Name: Ethan Paek

Date: 5/6/2020

Topic: COEN 140 Lab 6

Description: Implement Linear Regression and Ridge Regression in Python without using any machine learning libraries on the provided datasets.

Training set: http://www.cse.scu.edu/~yfang/coen140/crime-train.txt

Testing set: http://www.cse.scu.edu/~yfang/coen140/crime-test.txt

A description of the variables: http://www.cse.scu.edu/~yfang/coen140/communities.names

In [2]:
import numpy as np
import pandas as pd
import math

In [2]:
# load and store datasets from websites
train = pd.read_csv('data/crime-train.txt',delimiter='\t')
test = pd.read_csv('data/crime-test.txt',delimiter='\t')
train.head()

,ViolentCrimesPerPop,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn
0,0.67,-0.45,-1.85,-1.06,0.67,0.08,-0.85,-0.34,0.68,-0.24,...,-0.23,-0.02,-0.53,-1.08,-0.13,-0.66,-0.41,-0.56,1.26,-0.39
1,0.43,-0.45,-0.27,-0.22,-0.17,-0.34,-0.58,-0.50,-1.57,-0.29,...,-0.23,-0.33,-0.58,0.03,0.22,-0.46,-0.50,-0.11,-0.62,-0.39
2,0.12,-0.14,1.87,0.55,0.04,0.02,-1.19,-0.03,0.68,1.05,...,-0.23,-0.11,-1.51,1.07,0.07,-0.01,-0.41,0.77,0.52,-0.39
3,0.03,-0.38,0.53,-0.28,-0.79,-0.64,-0.35,-0.34,0.46,0.66,...,-0.23,-0.46,0.54,0.58,-0.08,-0.61,-0.23,-0.70,-0.62,-0.39
4,0.14,-0.30,-1.12,-0.74,-0.10,-0.40,-0.30,-0.19,0.68,0.76,...,-0.23,2.10,-0.92,-0.25,0.52,-0.06,-0.50,1.71,-0.27,-0.39


In [3]:
# store expected outcomes (1st column) into y_train; everything else for prediction goes into X_train
X_train = train.drop('ViolentCrimesPerPop',axis=1)
y_train = train['ViolentCrimesPerPop']

# we also have to convert values from string to float
X_train = np.float64(X_train)

# append 1s to the end of X_train as Dr. Fang mentioned in lectures
ones = np.ones(len(X_train))
X_train = np.column_stack((X_train, ones))

y_train = np.float64(y_train)
# turn the array into a matrix (with one column)
y_train = np.reshape(y_train, (-1,1))

# check to make sure we have the correct amount of communities and features
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (1595, 96)
y_train shape: (1595, 1)


In [4]:
# do the same thing from above but for the testing subset
X_test = test.drop('ViolentCrimesPerPop',axis=1)
y_test = test['ViolentCrimesPerPop']

X_test = np.float64(X_test)

ones = np.ones(len(X_test))
X_test = np.column_stack((X_test, ones))

y_test = np.float64(y_test)
# turn the array into a matrix (with one column)
y_test = np.reshape(y_test, (-1,1))

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_test shape: (399, 96)
y_test shape: (399, 1)


In [5]:
# this function simply computs the root mean square error (RMSE) by taking in two parameters: the predicted outcome matrix and the actual outcome matrix
def RMSE(prediction, actual):
    N = len(prediction)
    #some sorta issue here
    difference = prediction - actual
    total = 0
    
    for instance in difference:
        total += (instance ** 2)
        
    total_error = math.sqrt(total/N)
    return total_error

### Step 1: Perform ridge regression directly using the closed form solution. Use k-fold cross validate (k=5) to select the optimal 𝜆 parameter

In [6]:
# perform ridge regression and return list of predicted outcomes that each correspond to their actual values
def ridge_regression(x, y, test, lambda_value):
    left = np.linalg.inv(np.dot(x.T, x) + (lambda_value * np.identity(len(x.T))))
    right = np.dot(x.T, y)
    w = np.dot(left, right)
    
    prediction = []
    
    for Xtest in test:
        prediction.append(np.dot(Xtest.T, w))
        
    return prediction

In [138]:
lambda_value = 400
RMSEs = np.zeros((10,5))
minimum = 0

# do k-fold cross validation with k = 5
for segment in range(len(RMSEs)):
    for k in range(5):
        # train with 4/5 of our dataset
        X_train_k = np.concatenate((X_train[0:int(k*(len(X_train)/5))], X_train[int((k+1)*(len(X_train)/5)):int(len(X_train))]))
        y_train_k = np.concatenate((y_train[0:int(k*(len(y_train)/5))], y_train[int((k+1)*(len(y_train)/5)):int(len(y_train))]))
        
        # use the remaining 1/5 dataset to validate our results
        X_validate_k = X_train[int(k*(len(X_train)/5)):int((k+1)*(len(X_train)/5))]
        y_validate_k = y_train[int(k*(len(y_train)/5)):int((k+1)*(len(y_train)/5))]

        # perform ridge regression on each fold and each lambda 
        prediction = ridge_regression(X_train_k, y_train_k, X_validate_k, lambda_value)

        # for each lambda, put all 5 of the different RMSEs on the same row
        RMSEs[segment, k] = RMSE(prediction, y_validate_k)
        
    lambda_value /= 2

print("RMSE values:\n",RMSEs, "\n")
    
# get average RMSE for each lambda and choose optimal lambda
for i in range(len(RMSEs)-1):
    if (sum(RMSEs[i+1]) / 5) < (sum(RMSEs[minimum]) / 5):
        minimum = i+1

for j in range(len(RMSEs)):
    print("𝜆 =", (400 / (2 ** j)), ":", sum(RMSEs[j]) / 5)

optimal_lambda = 400 / (2 ** minimum)

print("\nThe optimal 𝜆 is:", optimal_lambda)
print("Training RMSE for ridge regression using closed form solution is", (sum(RMSEs[minimum]) / 5))

RMSE values:
 [[0.16031626 0.14530191 0.15755165 0.14669202 0.13770204]
 [0.1504947  0.13916403 0.14576515 0.13872299 0.12932324]
 [0.14641669 0.13761947 0.13961026 0.13589421 0.12684322]
 [0.14511564 0.13746282 0.1363316  0.13498737 0.1268823 ]
 [0.1449868  0.13758002 0.13455271 0.13464334 0.12781642]
 [0.14528919 0.13780086 0.13365482 0.13444721 0.12891979]
 [0.14570088 0.13815894 0.13329282 0.13430567 0.12988101]
 [0.14610098 0.13865628 0.13324682 0.13421881 0.1306319 ]
 [0.1464384  0.13923262 0.13336903 0.13420359 0.13119456]
 [0.14668497 0.13978827 0.13355288 0.13425675 0.13159847]] 

𝜆 = 400.0 : 0.14951277619075795
𝜆 = 200.0 : 0.14069401845317783
𝜆 = 100.0 : 0.13727677223186574
𝜆 = 50.0 : 0.1361559439275552
𝜆 = 25.0 : 0.13591585919851984
𝜆 = 12.5 : 0.13602237446230686
𝜆 = 6.25 : 0.13626786494076748
𝜆 = 3.125 : 0.13657095693642846
𝜆 = 1.5625 : 0.1368876407319742
𝜆 = 0.78125 : 0.13717626806866692

The optimal 𝜆 is: 25.0
Training RMSE for ridge regression using gradient descent is 0

Clearly, the best 𝜆 = 25, since the average RMSE for this value is lower than all other tested lambdas. In other words, the RMSE is minimized when the lambda is equal to 25 as opposed to any other lambda where 𝜆=400/(2^n) and 0 < n < 10

### Step 2: Perform linear regression using the gradient descent algorithm

In [113]:
# compute loss function for linear regression
def linear_gradient_descent(x, y, w):
    left = (y - np.dot(x, w)).T
    right = (y - np.dot(x, w))
    min_w = np.dot(left, right)
    return min_w

alpha = 5*(10**-5)

In [1]:
# use gradient descent until convergence to obtain w
L1 = 1
L0 = 0
# generate a Gaussian (normal) distribution for our initial value, based off how many features we have
w0 = np.random.normal(0, 1, (X_train.shape[1],1))

while (abs(L1 - L0) > (10**-5)):
    w1 = w0 - (alpha * np.dot(X_train.T, (np.dot(X_train, w0) - y_train)))
    L0 = linear_gradient_descent(X_train, y_train, w0)
    L1 = linear_gradient_descent(X_train, y_train, w1)
    w0 = w1

NameError: name 'np' is not defined

In [129]:
# this function takes in an NxK array and returns predictions in the form of an Nx1 array
def problem2(w_value, matrix):
    prediction = []
    for x in matrix:
        prediction.append(np.dot(w_value.T, x))
    return prediction

In [131]:
train_prediction = []
test_prediction = []

# make predictions and calculate RMSE for training data
train_prediction = problem2(w0, X_train)

train_linear_RSME = RMSE(train_prediction, y_train)
print("Training RMSE for linear regression using gradient descent is", train_linear_RSME)

# make predictions and calculate RMSE for test data
test_prediction = problem2(w0, X_test)

test_linear_RMSE = RMSE(test_prediction, y_test)
print("Testing RMSE for linear regression using gradient descent is", test_linear_RMSE)

Training RMSE for linear regression using gradient descent is 0.1279001630206582
Testing RMSE for linear regression using gradient descent is 0.145836802702037


### Comparing Error Rates

#### Lab 5

Training: 0.12768967421762187

Testing: 0.14583464490949133

#### Lab 6

Training: 0.1279001630206582

Testing: 0.145836802702037

### Step 3: Perform ridge regression with 5-fold cross validation using the gradient descent algorithm

In [86]:
# compute loss function for ridge regression
def ridge_gradient_descent(x, y, w, lambda_value):
    left = (y - np.dot(x, w)).T
    right = (y - np.dot(x, w))
    min_W = np.dot(left, right) + (optimal_lambda * np.dot(w0.T, w0))
    
    return min_W

alpha = 5*(10**-5)

In [118]:
# this function takes in an NxK array and returns predictions in the form of an Nx1 array
def problem3(w, validation):
    predictions = []
    for Xnew in validation:
        predictions.append(np.dot(w.T, Xnew))
    return predictions

In [137]:
# similar to problem 1, we need to find the optimal lambda
lambda_value = 400
RMSEs = np.zeros((10,5))
minimum = 0

# do k-fold cross validation with k = 5
for segment in range(len(RMSEs)):
    for k in range(5):
        # train with 4/5 of our dataset
        X_train_k = np.concatenate((X_train[0:int(k*(len(X_train)/5))], X_train[int((k+1)*(len(X_train)/5)):int(len(X_train))]))
        y_train_k = np.concatenate((y_train[0:int(k*(len(y_train)/5))], y_train[int((k+1)*(len(y_train)/5)):int(len(y_train))]))
        
        # use the remaining 1/5 dataset to validate our results
        X_validate_k = X_train[int(k*(len(X_train)/5)):int((k+1)*(len(X_train)/5))]
        y_validate_k = y_train[int(k*(len(y_train)/5)):int((k+1)*(len(y_train)/5))]
        
        # perform ridge regression using gradient descent on each fold and each lambda
        L1 = 1
        L0 = 0
        # generate a Gaussian (normal) distribution for our initial value, based off how many features we have
        w0 = np.random.normal(0, 1, (X_train.shape[1],1))
        while (abs(L1 - L0) > (10**-5)):
            w1 = w0 - (alpha * (np.dot(X_train_k.T, (np.dot(X_train_k, w0) - y_train_k)) + (lambda_value * w0)))
            L0 = ridge_gradient_descent(X_train_k, y_train_k, w0, lambda_value)
            L1 = ridge_gradient_descent(X_train_k, y_train_k, w1, lambda_value)
            w0 = w1
        
        validated_results = []
        validated_results = problem3(w0, X_validate_k)

        # for each lambda, put all 5 of the different RMSEs on the same row
        RMSEs[segment, k] = RMSE(validated_results, y_validate_k)
        
    lambda_value /= 2

print("RMSE values:\n",RMSEs, "\n")
    
# get average RMSE for each lambda and choose optimal lambda
for i in range(len(RMSEs)-1):
    if (sum(RMSEs[i+1]) / 5) < (sum(RMSEs[minimum]) / 5):
        minimum = i+1

for j in range(len(RMSEs)):
    print("𝜆 =", (400 / (2 ** j)), ":", sum(RMSEs[j]) / 5)

optimal_lambda = 400 / (2 ** minimum)

print("\nThe optimal 𝜆 is:", optimal_lambda)
print("Training RMSE for ridge regression using gradient descent is", (sum(RMSEs[minimum]) / 5))

RMSE values:
 [[0.16031645 0.14529659 0.15774393 0.14644346 0.1377206 ]
 [0.15041878 0.1391562  0.14576057 0.13872718 0.12955656]
 [0.14656764 0.13754041 0.1395291  0.13588014 0.12684171]
 [0.14511777 0.13735984 0.13693394 0.13483061 0.12691422]
 [0.1452592  0.1377219  0.13455798 0.13468397 0.1277978 ]
 [0.14502404 0.13841093 0.1335924  0.1344487  0.12898442]
 [0.14565418 0.13805621 0.13335367 0.13425258 0.12989125]
 [0.14623778 0.13838639 0.13314707 0.13455596 0.13030709]
 [0.14633717 0.13909454 0.134029   0.13433102 0.13090378]
 [0.1473196  0.14070872 0.13445381 0.13625563 0.1326497 ]] 

𝜆 = 400.0 : 0.1495042067724742
𝜆 = 200.0 : 0.1407238583845988
𝜆 = 100.0 : 0.13727179980315438
𝜆 = 50.0 : 0.136231274312913
𝜆 = 25.0 : 0.13600416890273567
𝜆 = 12.5 : 0.1360920985283161
𝜆 = 6.25 : 0.13624157733562461
𝜆 = 3.125 : 0.13652685691040173
𝜆 = 1.5625 : 0.1369391002923243
𝜆 = 0.78125 : 0.13827749133112024

The optimal 𝜆 is: 25.0
Training RMSE for ridge regression using gradient descent is 0.136

In [134]:
# re-compute w0 with the optimal lambda and all data from training subset so we can get results for our testing subset
L1 = 1
L0 = 0
# generate a Gaussian (normal) distribution for our initial value, based off how many features we have
w0 = np.random.normal(0, 1, (X_train.shape[1],1))
while (abs(L1 - L0) > (10**-5)):
    w1 = w0 - (alpha * (np.dot(X_train.T, (np.dot(X_train, w0) - y_train)) + (optimal_lambda * w0)))
    L0 = ridge_gradient_descent(X_train, y_train, w0, lambda_value)
    L1 = ridge_gradient_descent(X_train, y_train, w1, lambda_value)
    w0 = w1

test_predictions = []
test_predictions = problem3(w0, X_test)

test_ridge_RMSE = RMSE(test_predictions, y_test)
print("Testing RMSE for ridge regression using gradient descent is", test_ridge_RMSE)

Testing RMSE for ridge regression using gradient descent is 0.14550739450917766


### RMSE Values for Problem 3

Training: 0.13600416890273567

Testing: 0.14550739450917766